In [1]:
import numpy as np
import pandas as pd
#from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as mse
from sklearn.neighbors import KNeighborsRegressor as KNN
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
#from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import Normalizer

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('/Data_Science/Kurs_project_task/train.csv')

In [4]:
data.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [5]:
data.shape

(10000, 20)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.4+ MB


##### Нормализация Ecology* , Shops* и Helthcare*

In [7]:
data.loc[data['Ecology_2'] == 'A', 'Ecology_2'] = 0
data.loc[data['Ecology_2'] == 'B', 'Ecology_2'] = 1
data.loc[data['Ecology_3'] == 'A', 'Ecology_3'] = 0
data.loc[data['Ecology_3'] == 'B', 'Ecology_3'] = 1
data.loc[data['Shops_2'] == 'A', 'Shops_2'] = 0
data.loc[data['Shops_2'] == 'B', 'Shops_2'] = 1
data['Shops_1'] = data['Shops_1'] / 100
data['Social_1'] = data['Social_1'] / 100
data['Social_2'] = data['Social_2'] / 100000
data['Social_3'] = data['Social_3'] / 1000
data['Healthcare_1'].fillna(0, inplace = True)
data['Healthcare_1'] = data['Healthcare_1'] / 10000
data['Helthcare_2'] = data['Helthcare_2'] / 10

In [8]:
data.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,1,1,0.33,0.07976,0.005,0.0000,0.0,0.11,1,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,1,1,0.46,0.10309,0.001,0.0240,0.1,0.16,1,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,1,1,0.34,0.07759,0.000,0.0229,0.1,0.03,1,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,1,1,0.23,0.05735,0.003,0.1084,0.0,0.05,1,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,1,1,0.35,0.05776,0.001,0.2078,0.2,0.04,1,150226.531644


##### Очистка HouseYear

In [9]:
data.loc[(data['HouseYear'] < 1900) | (data['HouseYear'] > 2020), :]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
1497,10814,109,1.0,37.265070,20.239714,9.0,9,12.0,20052011,0.136330,1,1,0.30,0.06141,0.010,0.0262,0.3,0.06,1,254084.534396
4189,11607,147,2.0,44.791836,28.360393,5.0,4,9.0,4968,0.319809,1,1,0.25,0.04756,0.016,0.2857,0.5,0.08,1,243028.603096


In [10]:
data.loc[data['Id'] == 10814, 'HouseYear'] = 2011
data.loc[data['Id'] == 11607, 'HouseYear'] = 1968

In [11]:
data.loc[(data['HouseYear'] < 1900) | (data['HouseYear'] > 2020), :].shape[0]

0

##### Очистка Floor и HouseFloor

In [12]:
#data['HouseFloor'].value_counts().sort_index()

In [13]:
data.loc[(data['HouseFloor'] == 0) & (data['Floor'] > 0), :].shape[0]

269

In [14]:
data.loc[(data['HouseFloor'] == 0) & (data['Floor'] > 0), 'HouseFloor'] = data['Floor']

In [15]:
data.loc[(data['HouseFloor'] == 0) & (data['Floor'] > 0), :].shape[0]

0

In [16]:
data.loc[(data['Floor'] == 0) & (data['HouseFloor'] > 0), :].shape[0]

0

In [17]:
data.loc[(data['Floor'] > data['HouseFloor']), :].shape[0]

1556

In [18]:
data.loc[(data['Floor'] > data['HouseFloor']), 'Floor'] = data['HouseFloor']

In [ ]:
data.loc[(data['Floor'] > data['HouseFloor']), :].shape[0]

0

In [ ]:
data.loc[(data['HouseFloor'] > 50), :]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
6131,10806,5,1.0,51.944587,48.709601,1.0,6.0,99.0,1977,0.150818,1,1,0.16,0.03433,0.004,0.2643,0.4,0.05,1,296127.115515
8599,9300,74,2.0,71.747869,74.579809,9.0,5.0,99.0,1977,0.075779,1,1,0.06,0.01437,0.003,0.0000,0.0,0.02,1,243329.912579
8854,78,30,2.0,65.773749,66.811789,1.0,8.0,117.0,1977,0.000078,1,1,0.22,0.06398,0.141,0.1046,0.3,0.23,1,207248.370520


In [ ]:
data.loc[data['HouseFloor'] == 99, 'HouseFloor'] = 9
data.loc[data['HouseFloor'] == 117, 'HouseFloor'] = 11

In [ ]:
data.loc[(data['HouseFloor'] > 50), :].shape[0]

0

In [ ]:
#data['Floor'].value_counts().sort_index()

##### Очистка Square

In [ ]:
data['LifeSquare'].fillna(0, inplace = True)

In [ ]:
data.loc[(data['Square'] == 0) | (data['LifeSquare'] == 0), :].shape[0]

2113

In [ ]:
data.loc[(data['Square'] == 0) & (data['LifeSquare'] > 0), 'Square'] = data['LifeSquare']
data.loc[(data['LifeSquare'] == 0) & (data['Square'] > 0), 'LifeSquare'] = data['Square']

In [ ]:
data.loc[(data['Square'] == 0) | (data['LifeSquare'] == 0), :].shape[0]

0

In [ ]:
data.loc[(data['Square'] >= data['LifeSquare']) & (data['LifeSquare'] > 0), :].shape[0]

9518

In [ ]:
data.loc[(data['LifeSquare'] > data['Square']), :].shape[0]

482

In [ ]:
data.loc[(data['LifeSquare'] > data['Square']) & (data['LifeSquare'] < 650), 'Square'] = data['LifeSquare']
data.loc[(data['LifeSquare'] > data['Square']), :]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
4328,16550,27,3.0,81.694417,7480.592129,1.0,9.0,17.0,2016,0.017647,1,1,0.02,0.00469,0.0,0.0,0.0,0.0,1,217357.492366


In [ ]:
data.loc[data['Square'] < 15, 'Square'] = data['Square'] * 10

In [ ]:
data.loc[data['Square'] < 20, :]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
1891,13685,0,1.0,17.498542,13.446544,0.0,5.0,5.0,1961,0.319809,1,1,0.25,0.04756,0.016,0.2857,0.5,0.08,1,88871.750230
3794,523,101,1.0,18.509177,15.170851,3.0,5.0,5.0,1970,0.225825,0,1,0.41,0.06558,0.007,0.1548,0.3,0.02,1,115845.861527
4739,12676,81,3.0,15.988889,15.988889,7.0,4.0,5.0,1960,0.319809,1,1,0.25,0.04756,0.016,0.2857,0.5,0.08,1,78388.806186
5617,15877,57,3.0,16.698571,16.698571,8.0,5.0,5.0,1962,0.133215,1,1,0.49,0.11395,0.003,0.1406,0.3,0.04,0,77835.185808
6945,12927,62,2.0,17.414181,17.414181,1.0,4.0,20.0,1977,0.072158,1,1,0.02,0.00629,0.001,0.0000,0.0,0.00,0,180750.471749
7767,6927,21,1.0,16.117154,8.822988,0.0,2.0,5.0,1965,0.194489,1,1,0.47,0.08004,0.003,0.0125,0.3,0.05,1,79913.267856
7897,10635,21,1.0,18.983962,11.419796,2.0,1.0,5.0,1967,0.194489,1,1,0.47,0.08004,0.003,0.0125,0.3,0.05,1,112411.221700
9516,14236,0,1.0,19.927423,15.541536,0.0,2.0,5.0,1962,0.319809,1,1,0.25,0.04756,0.016,0.2857,0.5,0.08,1,102395.788856


##### Очистка Price

In [ ]:
data.loc[data['Id'] == 7243, 'Price'] = data['Price'] + 100000
data.loc[data['Id'] == 13237, 'Price'] = data['Price'] + 100000

#### Построение моделей

In [ ]:
train, valid = train_test_split(data, test_size=0.3, random_state=42)

In [ ]:
train.shape, valid.shape

((7000, 20), (3000, 20))

In [ ]:
data.columns

Index(['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Ecology_2',
       'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1',
       'Helthcare_2', 'Shops_1', 'Shops_2', 'Price'],
      dtype='object')

In [ ]:
fts = ['DistrictId', 'Square', 'Floor', 'HouseYear', 'Ecology_1', 'Ecology_2', 'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1', 'Helthcare_2', 'Shops_1', 'Shops_2']

### KNN_Regressor

In [ ]:
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'svg'
%matplotlib inline

In [ ]:
knn_vals = np.arange(2, 40, 2)

In [ ]:
r2_valid_knn = []
r2_train_knn = []
mse_valid_knn = []
mse_train_knn = []
for val in knn_vals:
#    klr = KNN(n_neighbors=val, weights='distance')
    klr = KNN(n_neighbors=val)
    klr.fit(train.loc[:, fts], train['Price'])
    klr_pred = klr.predict(train.loc[:, fts])
    klr_pred_valid = klr.predict(valid.loc[:, fts])
    r2_train_knn.append(r2(train['Price'], klr_pred ))
    r2_valid_knn.append(r2(valid['Price'], klr_pred_valid))
    mse_train_knn.append(mse(train['Price'], klr_pred))
    mse_valid_knn.append(mse(valid['Price'], klr_pred_valid))

In [ ]:
plt.plot(knn_vals, r2_train_knn)
plt.plot(knn_vals, r2_valid_knn)
plt.xlabel('n_neighbors')
plt.ylabel('R2')
plt.legend(['train', 'valid'])
plt.show()

In [ ]:
plt.plot(knn_vals, mse_train_knn)
plt.plot(knn_vals, mse_valid_knn)
plt.xlabel('n_neighbors')
plt.ylabel('MSE')
plt.legend(['train', 'valid'])
plt.show()

### RandomForestRegressor

In [ ]:
rfr_vals = np.arange(1)

In [ ]:
rfr = RF(random_state=42)
r2_valid_rfr = []
r2_train_rfr = []
mse_valid_rfr = []
mse_train_rfr = []
for val in rfr_vals:
    rfr = RF(n_estimators=300, max_depth=20, min_samples_leaf=2, random_state=42)
    rfr.fit(train.loc[:, fts], train['Price'])
    rfr_pred = rfr.predict(train.loc[:, fts])
    rfr_pred_valid = rfr.predict(valid.loc[:, fts])
    r2_train_rfr.append(r2(train['Price'], rfr_pred ))
    r2_valid_rfr.append(r2(valid['Price'], rfr_pred_valid))
    mse_train_rfr.append(mse(train['Price'], rfr_pred))
    mse_valid_rfr.append(mse(valid['Price'], rfr_pred_valid))

In [ ]:
plt.scatter(rfr_vals, r2_train_rfr)
plt.scatter(rfr_vals, r2_valid_rfr)
plt.xlabel('n_estimators')
plt.ylabel('R2')
plt.legend(['train', 'valid'])
plt.show()

In [ ]:
plt.scatter(rfr_vals, mse_train_rfr)
plt.scatter(rfr_vals, mse_valid_rfr)
plt.xlabel('n_estimators')
plt.ylabel('MSE')
plt.legend(['train', 'valid'])
plt.show()

### Выполняю предсказание с помощью RandomForestRegressor

In [ ]:
data_test = pd.read_csv('/Data_Science/Kurs_project_task/test.csv')

In [ ]:
data_test.info()

In [ ]:
data_test.head()

##### Нормализация Ecology* , Shops* и Helthcare*

In [ ]:
data_test.loc[data_test['Ecology_2'] == 'A', 'Ecology_2'] = 0
data_test.loc[data_test['Ecology_2'] == 'B', 'Ecology_2'] = 1
data_test.loc[data_test['Ecology_3'] == 'A', 'Ecology_3'] = 0
data_test.loc[data_test['Ecology_3'] == 'B', 'Ecology_3'] = 1
data_test.loc[data_test['Shops_2'] == 'A', 'Shops_2'] = 0
data_test.loc[data_test['Shops_2'] == 'B', 'Shops_2'] = 1
data_test['Shops_1'] = data_test['Shops_1'] / 100
data_test['Social_1'] = data_test['Social_1'] / 100
data_test['Social_2'] = data_test['Social_2'] / 100000
data_test['Social_3'] = data_test['Social_3'] / 1000
data_test['Healthcare_1'].fillna(0, inplace = True)
data_test['Healthcare_1'] = data_test['Healthcare_1'] / 10000
data_test['Helthcare_2'] = data_test['Helthcare_2'] / 10

In [ ]:
data_test.head()

##### Очистка HouseFloor

In [ ]:
data_test.loc[(data_test['HouseFloor'] == 0) & (data_test['Floor'] > 0), :].shape[0]

In [ ]:
data_test.loc[(data_test['HouseFloor'] == 0) & (data_test['Floor'] > 0), 'HouseFloor'] = data_test['Floor']

In [ ]:
data_test.loc[(data_test['HouseFloor'] == 0) & (data_test['Floor'] > 0), :].shape[0]

In [ ]:
data_test.loc[(data_test['Floor'] == 0) & (data_test['HouseFloor'] > 0), :].shape[0]

In [ ]:
data_test.loc[(data_test['Floor'] > data_test['HouseFloor']), :].shape[0]

In [ ]:
data_test.loc[(data_test['Floor'] > data_test['HouseFloor']), 'Floor'] = data_test['HouseFloor']

In [ ]:
data_test.loc[(data_test['Floor'] > data_test['HouseFloor']), :].shape[0]

In [ ]:
data_test['Floor'].value_counts().sort_index()

##### Очистка Square

In [ ]:
data_test['LifeSquare'].fillna(0, inplace = True)

In [ ]:
data_test.loc[(data_test['Square'] == 0) | (data_test['LifeSquare'] == 0), :].shape[0]

In [ ]:
data_test.loc[(data_test['Square'] == 0) & (data_test['LifeSquare'] > 0), 'Square'] = data_test['LifeSquare']
data_test.loc[(data_test['LifeSquare'] == 0) & (data_test['Square'] > 0), 'LifeSquare'] = data_test['Square']

In [ ]:
data_test.loc[(data_test['Square'] == 0) | (data_test['LifeSquare'] == 0), :].shape[0]

In [ ]:
data_test.loc[(data_test['Square'] >= data_test['LifeSquare']) & (data_test['LifeSquare'] > 0), :].shape[0]

In [ ]:
data_test.loc[(data_test['LifeSquare'] > data_test['Square']), :].shape[0]

In [ ]:
data_test.loc[(data_test['LifeSquare'] > data_test['Square']) & (data_test['LifeSquare'] < 200), 'Square'] = data_test['LifeSquare']
data_test.loc[(data_test['LifeSquare'] > data_test['Square']), :]

In [ ]:
data_test.loc[data_test['Square'] < 13, 'Square'] = data_test['Square'] * 10

In [ ]:
data_test.loc[(data_test['Square'] < 20), :]

##### Предсказание с помощью обученной модели

In [ ]:
pred_test = rfr.predict(data_test.loc[:, fts])
pred_test.shape

In [ ]:
data_test['Price'] = pred_test

In [ ]:
data_test.head()

In [ ]:
data_test.loc[:, ['Id', 'Price']].to_csv('/Data_Science/Shulzhenkoas_predictions.csv', index=None)